# Ollama + OpenAI + Python

## 1. Вкажіть назву моделі

Якщо ви завантажили іншу модель, ніж "phi3:mini", змініть значення в комірці нижче. Ця змінна буде використовуватися в коді протягом усього блокнота.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Налаштування клієнта Open AI

Зазвичай клієнт OpenAI використовується з OpenAI.com або Azure OpenAI для взаємодії з великими мовними моделями.  
Однак його також можна використовувати з Ollama, оскільки Ollama надає сумісну з OpenAI кінцеву точку за адресою "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Створення завершення чату

Тепер ми можемо використовувати OpenAI SDK для створення відповіді на розмову. Цей запит має створити хайку про котів:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Інженерія запитів

Перше повідомлення, яке надсилається мовній моделі, називається "системним повідомленням" або "системним запитом", і воно встановлює загальні інструкції для моделі.  
Ви можете надати власний користувацький системний запит, щоб спрямувати мовну модель на створення вихідних даних у іншому стилі.  
Змініть `SYSTEM_MESSAGE` нижче, щоб модель відповідала, як ваш улюблений персонаж із фільму чи серіалу, або знайдіть натхнення для інших системних запитів на [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Після того, як ви налаштуєте системне повідомлення, надайте перше запитання користувача у `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Приклади з кількома зразками

Ще один спосіб спрямувати мовну модель — надати "кілька зразків", тобто послідовність прикладів запитань/відповідей, які демонструють, як вона повинна реагувати.

У наведеному нижче прикладі модель намагається діяти як асистент викладача, надаючи кілька прикладів запитань і відповідей, які міг би дати асистент, а потім пропонує моделі запитання, яке міг би поставити студент.

Спробуйте спочатку, а потім змініть `SYSTEM_MESSAGE`, `EXAMPLES` і `USER_MESSAGE` для нового сценарію.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Генерація з використанням пошуку

RAG (Retrieval Augmented Generation) — це техніка, яка дозволяє мовній моделі точно відповідати на запитання для конкретної області, спочатку отримуючи релевантну інформацію з джерела знань, а потім генеруючи відповідь на основі цієї інформації.

Ми надали локальний CSV-файл із даними про гібридні автомобілі. Код нижче читає CSV-файл, шукає збіги із запитанням користувача, а потім генерує відповідь на основі знайденої інформації. Зверніть увагу, що це займе більше часу, ніж будь-який із попередніх прикладів, оскільки модель отримує більше даних. Якщо ви помітили, що відповідь все ще не базується на даних, ви можете спробувати системну інженерію або інші моделі. Загалом, RAG є більш ефективним або з більшими моделями, або з тонко налаштованими версіями SLM.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
